In [1]:
import pandas as pd

In [2]:
# get the constallation name abbreviations, to limit variable stars to sensible names
def load_const_file(const_file):
#const_file = 'constellation_names.txt'
    f_const = open(const_file)
    line = ' '
    const_abbr = []
    while line != '':
        line = f_const.readline()
        const_abbr.append(line.split('\t')[-1].replace('\n',''))
    const_abbr = const_abbr[2:]
    f_const.close()
    const_abbr_low = [ const.lower() for const in const_abbr ]
    return const_abbr_low

In [3]:
def load_seq_pattern(seq_file):
    '''
    get the sequence files available on BAA and return as regexp pattern

    '''
    seq = pd.read_csv('baa_sequences.txt',delimiter='\t')
    obj = seq['Object Id']
    display(obj)
    obj.describe()
    obs_lower = obj.str.lower()
    seq_pattern = '|'.join(obs_lower)
    return seq_pattern
    

In [4]:
in_file = '/home/john/code/Astro/variable_star/all_star_summary.csv'

In [5]:
# read in BAA variable star summary data as a Panda Data Frame
vsdb_full = pd.read_csv(in_file)
vsdb_full.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/john/astro/variable_star_data/baa/all_star_summary.csv'

In [ ]:
vsdb_full.describe()

In [ ]:
vsdb_full.hist('Number of Observations', bins=100)

In [ ]:
# can also call describe() on a Series object
vsdb_full['Maximum Magnitude'].describe()

In [ ]:
vsdb_full[['Variable Star', 'Number of Observations', 'Number of Observers']].sort_values(by='Number of Observations', ascending=False).head(30)

In [ ]:
# create a pattern to search which is an OR of each element in const_abbr (i.e. any constallation abbreviation)
const_abbrev = load_const_file('constellation_names.txt')
pattern = '|'.join(const_abbrev)
# need to watch the trailing OR, which seems to cause everything to match
pattern = pattern[:-1]
print(pattern)

In [ ]:
# I only want the stars where there are two fields in the name (e.g. 'TX Dra')
vsdb_allconst = vsdb_full
vsdb_allconst['Valid_nFields'] = vsdb_allconst['Variable Star'].str.count(' ')

#remove rows which don't have 1 nFields
vsdb_allconst.drop(vsdb_allconst[vsdb_allconst.Valid_nFields != 1].index, inplace=True)

# set the second field to be a 'Constellation' description.  Needs to be lower case to match pattern, above.
vs_const = vsdb_allconst['Variable Star'].str.split(expand=True)
vsdb_allconst['VS Constellation'] = vs_const[1].str.lower()

In [ ]:
# check whether str.contains is working...
vsdb_allconst['VS Constellation'].str.contains(pattern)==True

In [ ]:
# drop all rows where the conditional statement 
vsdb_allconst.drop(vsdb_allconst[vsdb_allconst['VS Constellation'].str.contains(pattern)==False].index, inplace=True)
vsdb_allconst.head()

In [ ]:
# vsdb, after initial filter on valid star name (two fields, contains a constellation abbreviation)
vsdb_allconst.describe()

In [ ]:
vsdb_allconst[['VS Constellation']].value_counts()

In [ ]:
# slicing via []
vsdb_allconst[vsdb_allconst['VS Constellation']=='cyg']

In [ ]:
#  this conditional statement returns a pandas Series of booleans
vsdb_allconst['VS Constellation']=='cyg'

In [ ]:
type(vsdb_allconst['VS Constellation']=='cyg')

In [ ]:
# check the dtype. Note the () are needed to prevent dtype operating on 'cyg'
(vsdb_allconst['VS Constellation']=='cyg').dtype

In [ ]:
# slicing via loc
# this is equivalent (I think!) to 
# vsdb[vsdb['VS Constellation']=='cyg']
vsdb_allconst.loc[vsdb_allconst['VS Constellation']=='cyg']

In [ ]:
vsdb_allconst[vsdb_allconst['VS Constellation']=='cyg'].sort_values(by='Number of Observations', ascending=False)

In [ ]:
# Having done that, now look at TARGET CONSTELLATIONS, but selecting the constallations that I 
# want to observe
# create a pattern to search which is an OR of each element in const_abbr (i.e. any constallation abbreviation)
vsdb_targets = vsdb_allconst
const_abbrev = load_const_file('constellation_names_targets.txt')
pattern = '|'.join(const_abbrev)
# need to watch the trailing OR, which seems to cause everything to match
pattern = pattern[:-1]
print(pattern)
# drop all rows where the conditional statement 
vsdb_targets.drop(vsdb_targets[vsdb_targets['VS Constellation'].str.contains(pattern)==False].index, inplace=True)
# start filtering on conditions
# first, range > v1.0
# this reduces the number of objects from ~2100 to ~1200
vsdb_targets = vsdb_targets[vsdb_targets['Range']>=1.0]
vsdb_targets.head()

In [ ]:
# start filtering on conditions
# first, range > v1.0
# this reduces the number of objects from ~2100 to ~1200
vsdb_targets = vsdb_targets[vsdb_targets['Range']>=1.0]
vsdb_targets.describe()

In [ ]:
# filter on min magnitude to find suitable binocular targets
#  reduces targets to ~200
vsdb_targets = vsdb_targets[vsdb_targets['Minimum Magnitude']<=9.0]
vsdb_targets.describe()

In [ ]:
vsdb_targets['Number of Observations'].hist()

In [ ]:
vsdb_targets.describe()

In [26]:
# generate a target list from the 25% least observed variables
vs_target_list = vsdb_targets[vsdb_targets['Number of Observations']<=275]
# although probably better to try filtering on EXISTING baa sequence file first
# then possibly time since last obs?

In [27]:
vs_target_list.drop(columns='Valid_nFields')

,Variable Star,Number of Observations,Number of Observers,First Observation JD / UT,Latest Observation JD / UT,Maximum Magnitude,Minimum Magnitude,Range,VS Constellation
93,ALPHA UMI,3,1,2453716.356000 11 Dec 2005 20:32:38.400000,2454465.519000 31 Dec 2007 00:27:21.600000,0.10,2.80,2.70,umi
189,BETA LEO,67,5,2443571.517000 4 Mar 1978 00:24:28.800000,2458951.469000 11 Apr 2020 23:15:21.600000,1.60,3.00,1.40,leo
227,BM ORI,5,1,2454498.263000 1 Feb 2008 18:18:43.200000,2457421.279000 2 Feb 2016 18:41:45.600000,5.50,8.50,3.00,ori
320,CO AUR,229,5,2442427.437000 14 Jan 1975 22:29:16.800000,2454829.512500 29 Dec 2008 00:18:00.000000,7.30,8.30,1.00,aur
342,CT DEL,69,3,2456710.729000 22 Feb 2014 05:29:45.600000,2458960.615000 21 Apr 2020 02:45:36.000000,6.90,8.20,1.30,del
372,DELTA UMA,217,7,2440420.524000 18 Jul 1969 00:34:33.600000,2459050.417000 19 Jul 2020 22:00:28.800000,2.10,4.40,2.30,uma
384,DK BOO,9,1,2458245.448000 6 May 2018 22:45:07.200000,2459310.347000 5 Apr 2021 20:19:40.800000,7.70,8.70,1.00,boo
392,DN GEM,12,1,2419477.330000 15 Mar 1912 19:55:12.000000,2419540.420000 17 May 1912 22:04:48.000000,4.00,7.20,3.20,gem
443,EK CEP,221,3,2446491.388000 1 Mar 1986 21:18:43.200000,2458961.578000 22 Apr 2020 01:52:19.200000,7.60,8.90,1.30,cep
452,EP Aqr,16,1,2455854.309000 19 Oct 2011 19:24:57.600000,2456272.239000 10 Dec 2012 17:44:09.600000,6.30,7.30,1.00,aqr


In [28]:
vs_target_list.sort_values('VS Constellation')

,Variable Star,Number of Observations,Number of Observers,First Observation JD / UT,Latest Observation JD / UT,Maximum Magnitude,Minimum Magnitude,Range,Valid_nFields,VS Constellation
1895,V1229 Aql,2,1,2440695.626000 19 Apr 1970 03:01:26.400000,2440709.580000 3 May 1970 01:55:12.000000,7.30,8.60,1.30,1,aql
452,EP Aqr,16,1,2455854.309000 19 Oct 2011 19:24:57.600000,2456272.239000 10 Dec 2012 17:44:09.600000,6.30,7.30,1.00,1,aqr
1201,RT AUR,50,5,2440287.326000 6 Mar 1969 19:49:26.400000,2451890.588000 12 Dec 2000 02:06:43.200000,4.90,6.10,1.20,1,aur
699,IU AUR,58,3,2449597.558000 2 Sep 1994 01:23:31.200000,2458482.338800 29 Dec 2018 20:07:52.320000,6.90,9.00,2.10,1,aur
2495,WW AUR,202,6,2443549.312000 9 Feb 1978 19:29:16.800000,2458469.742000 17 Dec 2018 05:48:28.800000,5.40,6.76,1.36,1,aur
2639,ZETA AUR,226,4,2449720.620800 3 Jan 1995 02:53:57.120000,2458917.396000 8 Mar 2020 21:30:14.400000,2.90,4.20,1.30,1,aur
2141,V428 AUR,235,4,2458099.330000 11 Dec 2017 19:55:12.000000,2459906.392000 22 Nov 2022 21:24:28.800000,6.50,8.10,1.60,1,aur
320,CO AUR,229,5,2442427.437000 14 Jan 1975 22:29:16.800000,2454829.512500 29 Dec 2008 00:18:00.000000,7.30,8.30,1.00,1,aur
384,DK BOO,9,1,2458245.448000 6 May 2018 22:45:07.200000,2459310.347000 5 Apr 2021 20:19:40.800000,7.70,8.70,1.00,1,boo
2452,VZ CAM,182,7,2440285.348000 4 Mar 1969 20:21:07.200000,2457626.500000 26 Aug 2016 00:00:00.000000,4.60,5.60,1.00,1,cam


In [29]:
vs_target_list['VS Constellation'].value_counts()

aur    6
cyg    5
umi    3
oph    3
cep    3
del    3
ori    3
mon    2
gem    2
vul    2
peg    2
cas    2
cet    2
tau    1
aql    1
psc    1
lep    1
her    1
cap    1
cma    1
leo    1
per    1
lib    1
com    1
aqr    1
boo    1
uma    1
cam    1
Name: VS Constellation, dtype: int64